Final Project Check-in 2018-11-16
------

Group Name
-----

We are Groot

Student Names
----

1. Jialiang Shi
2. Julia Tavares
3. Wei Wei
4. Xiao Han
5. Xueyin(Jessica) Wang

Load Data
-----

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
credits = pd.read_csv('dataset/tmdb_5000_credits.csv')
movies = pd.read_csv('dataset/tmdb_5000_movies_nonull.csv')

### Merge two datasets using id

In [3]:
movies_sub = movies.loc[:, ['id', 'original_title', 'genres', 'keywords', 'budget']]
credits_sub = credits.loc[:, ['movie_id', 'cast']].rename(columns={'movie_id':'id'})
df = movies_sub.merge(credits_sub)

In [4]:
# drop movies with no genre info
for i, row in df.iterrows():
    if row['genres'] == '[]':
        df.drop(i, inplace=True)
df = df.dropna()

In [5]:
# All cast: # of movies
cast_all = defaultdict(int)

for item in df.cast:
    for p in json.loads(item):
        cast_all[p['name']] += 1

In [6]:
df.head(5)

,id,original_title,genres,keywords,budget,cast
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",237000000,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",300000000,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",245000000,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",250000000,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",260000000,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


### Modify df

In [7]:
def extract_gender(cell):
    """Extract cast gender"""
    female = 0
    male = 0
    for item in json.loads(cell):
        if item['gender'] == 1: female += 1
        elif item['gender'] == 2: male += 1
        else: continue
            
    return female, male

def larger_n(col, n):
    """filter the column"""
    keywords = defaultdict(int)
    for row in df[col]:
        for entry in row:
            keywords[entry] += 1
    kw_cnt = sorted(keywords.items(), key=lambda x: -x[1])
    return [kw[0] for kw in kw_cnt if kw[1] >= n]

def modify_df(cell):
    """Convert string-format json cell to list"""
    my_list = []
    for item in json.loads(cell):
        my_list.append(item['name'])
 
    return my_list

def concat_names(cell):
    """Concatenate first names and last names"""
    names = []
    for name in cell:
        names.append(name.replace(' ', ''))
    return names

In [8]:
for col_name in ['genres', 'cast', 'keywords']:
    if col_name == 'cast':
        gen = df[col_name].apply(extract_gender)
        df['female_pct'] = gen.apply(lambda x: x[0]/(x[0]+x[1]+0.001))
        df['male_pct'] = gen.apply(lambda x: x[1]/(x[0]+x[1]+0.001))
    
    df[col_name] = df[col_name].apply(modify_df)

In [9]:
# drop cast who has only one movie
cast_to_remove = [key for (key, value) in cast_all.items() if value == 1]
for i, row in df.iterrows():
    df.at[i, 'cast'] = list(set(row.cast) - set(cast_to_remove))
    
# concatenate first and last names
df['cast'] = df['cast'].apply(concat_names)

In [10]:
larger_kw = larger_n('keywords', 30)
df['keywords'] = df['keywords'].apply(lambda cell: [kw for kw in cell if kw in larger_kw])

In [11]:
df.head(5)

,id,original_title,genres,keywords,budget,cast,female_pct,male_pct
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[future, alien, soldier, battle, 3d]",237000000,"[SigourneyWeaver, KevinDorman, WoodySchultz, C...",0.374988,0.624980
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[love of one's life, ship, aftercreditsstinger]",300000000,"[VanessaBranch, KevinMcNally, PeterDonaldBadal...",0.166661,0.833306
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, sequel]",245000000,"[MonicaBellucci, DanielCraig, AndrewScott, Jes...",0.191174,0.808812
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[terrorist, secret identity, superhero]",250000000,"[GaryOldman, JamesQuinn, MichaelPower, AidanGi...",0.130951,0.869037
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, alien, escape, 3d]",260000000,"[JoshDaugherty, JamesEmbree, BryanCranston, Jo...",0.239990,0.759970


Fit scikit-learn model
----

We so far have tried two different models using "cast + gender proportion" and "keywords + budget" respectively as features.

### Cast+gender vs Genres

In [12]:
for i, row in df.iterrows():
    df.at[i, 'cast'] = ' '.join(row['cast'])

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [14]:
# One-Hot encoding
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit_transform(df.genres)
train_data, test_data, train_labels, test_labels = train_test_split(df.loc[:, ['cast', 'female_pct', 'male_pct']], 
                                                                    mlb.transform(df.genres),
                                                                    test_size=0.2, random_state=0)

In [15]:
# Vectorization
vectorizer = CountVectorizer(ngram_range=(1, 2))
train_cast_vect = vectorizer.fit_transform(train_data.iloc[:, 0])
test_cast_vect = vectorizer.transform(test_data.iloc[:, 0])

train_features = np.concatenate((train_cast_vect.toarray(), np.asarray(train_data.iloc[:, 1:])), axis=1)
test_features = np.concatenate((test_cast_vect.toarray(), np.asarray(test_data.iloc[:, 1:])), axis=1)

In [16]:
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(MultinomialNB())

clf.fit(train_features, train_labels)
predicted = clf.predict(test_features)

### Keywords+budget vs Genres

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb1 = MultiLabelBinarizer()
genres_dummies = pd.DataFrame(mlb1.fit_transform(
    df['genres']), columns=mlb1.classes_).add_prefix('genre_')
keywords_dummies = pd.DataFrame(mlb1.fit_transform(
    df['keywords']), columns=mlb1.classes_).add_prefix('kw_')
df_new = pd.concat([
    genres_dummies, keywords_dummies], axis=1)

df_y = df_new.loc[:, df_new.columns.str.startswith('genre_')]
df_X = df_new.drop(
    df_new.columns[df_new.columns.str.startswith('genre_')], inplace=False, axis=1)

In [18]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.33, random_state=42)

In [19]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier

classifier = BinaryRelevance(RandomForestClassifier(n_estimators=10))
classifier.fit(train_X, train_y)
predictions = classifier.predict(test_X)

accuracy_score(test_y, predictions)

0.07301587301587302

In [25]:
predictions_dense = predictions.todense()

for i in range(predictions.shape[1]):
    print('{}: {}'.format(mlb.classes_[i], 
                          accuracy_score(np.asarray(test_y)[:, i], np.asarray(predictions_dense)[:, i])))

Action: 0.7911111111111111
Adventure: 0.8520634920634921
Animation: 0.9473015873015873
Comedy: 0.6673015873015873
Crime: 0.8438095238095238
Documentary: 0.9765079365079365
Drama: 0.6082539682539683
Family: 0.8926984126984127
Fantasy: 0.9053968253968254
Foreign: 0.9911111111111112
History: 0.9580952380952381
Horror: 0.8882539682539683
Music: 0.9517460317460318
Mystery: 0.8971428571428571
Romance: 0.8025396825396826
Science Fiction: 0.9123809523809524
TV Movie: 0.9987301587301587
Thriller: 0.7263492063492063
War: 0.9701587301587301
Western: 0.986031746031746


Evaluation Metric
----

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

### Overall accuracy, f1_score, precision, and recall

In [22]:
accuracy = accuracy_score(y_pred=predicted, y_true=np.array(test_labels))
f1 = f1_score(y_pred=predicted, y_true=np.array(test_labels), average='micro')
precision = precision_score(y_pred=predicted, y_true=np.array(test_labels), average='micro')
recall = recall_score(y_pred=predicted, y_true=np.array(test_labels), average='micro')

print("Subset Accuracy: {}".format(accuracy)) # subset accuracy
print("Global F1 score: {}".format(f1))
print("Global Precision: {}".format(precision))
print("Global Recall: {}".format(recall))

Subset Accuracy: 0.1382198952879581
Global F1 score: 0.449271377508936
Global Precision: 0.6819699499165276
Global Recall: 0.33497334973349735


### By class: f1_score, precision, and recall

Our goal is to predict the movie genres, so recall is more important for us. From the classification report below, we can see that only **Drama**, **Comedy**, and maybe Action as well, has relatively good recall score.

In [23]:
print(classification_report(test_labels, predicted, target_names=mlb.classes_))

                 precision    recall  f1-score   support

         Action       0.63      0.41      0.49       213
      Adventure       0.59      0.23      0.33       150
      Animation       1.00      0.21      0.35        52
         Comedy       0.75      0.63      0.68       348
          Crime       0.71      0.07      0.13       139
    Documentary       1.00      0.05      0.10        19
          Drama       0.72      0.67      0.69       474
         Family       0.82      0.24      0.37        96
        Fantasy       0.64      0.11      0.19        82
        Foreign       0.00      0.00      0.00         7
        History       0.40      0.05      0.09        41
         Horror       0.78      0.06      0.12       110
          Music       0.50      0.03      0.06        32
        Mystery       0.00      0.00      0.00        75
        Romance       0.60      0.12      0.19       182
Science Fiction       0.55      0.10      0.17       106
       TV Movie       0.00    

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


### By class: accuracy score

Accuracy is not a good evaluation metrics for our model, especially for some of the movie genres that are not commonly seen. For example, for the "TV Movie" class, the accuracy score is amazingly high, 0.9989, but the reality is that we only have 8 movies labeled as "TV Movie" among all 4500 movies. So the accuracy scores, though they look pretty good, cannot deliver much useful information of the performance of our model.

In [24]:
for i in range(predicted.shape[1]):
    print('{}: {}'.format(mlb.classes_[i], 
                          accuracy_score(np.asarray(test_labels)[:, i], np.asarray(predicted)[:, i])))

Action: 0.8136125654450261
Adventure: 0.8544502617801047
Animation: 0.9570680628272251
Comedy: 0.7884816753926701
Crime: 0.8607329842931937
Documentary: 0.981151832460733
Drama: 0.7078534031413612
Family: 0.9183246073298429
Fantasy: 0.9183246073298429
Foreign: 0.9926701570680628
History: 0.956020942408377
Horror: 0.8900523560209425
Music: 0.9664921465968587
Mystery: 0.9193717277486911
Romance: 0.8167539267015707
Science Fiction: 0.8910994764397906
TV Movie: 0.9989528795811519
Thriller: 0.7225130890052356
War: 0.9654450261780104
Western: 0.9832460732984293
